# Aeroelastic Polar and Torsional Divergence of a 2D Airfoil Section

---

Hello! Welcome to **Introduction to Aeroelastic Instabilities with Jupyter Notebooks**. This is a practical, hands-on module designed for a 6-hour PhD short course taught by Francesco M. A. Mitrotta at **Politecnico di Bari** in January 2026. The course is intended for doctoral students from diverse engineering backgrounds, you don't need prior advanced aerospace engineering knowledge! We assume only basic knowledge of mechanics (forces, moments, springs) and some familiarity with differential equations. If you're comfortable with the concept of equilibrium and Hooke's law, you're ready to start.

This practical module takes a computational approach to understanding aeroelastic phenomena. Rather than spending lectures on mathematical derivations, we'll use Python and Jupyter notebooks to visualize and explore aeroelastic instabilities. The course follows a "learn by doing" philosophy, and students who don't know Python will learn as we work through the notebooks together.

This Jupyter notebook will guide you through the fundamentals of aeroelastic coupling using the **typical section model**. We'll start by deriving the aeroelastic polar, in other words how the lift force is affected by the elastic deformation of the wing, and build the foundation for understanding critical phenomena like **divergence** and **flutter**. We're going to start with physical intuition and concrete examples. Don't worry if you don't grasp every mathematical detail immediately; we'll progressively build understanding through interactive visualizations and hands-on exploration.

For best results, after you work through this notebook, experiment with the plots, modify the parameters, and observe how the system behavior changes. Try to develop physical intuition before moving to the next topic.

To execute this Notebook, we assume you're running it through Binder or Google Colab, or that you have invoked the notebook server using: `jupyter notebook`.

## Learning Objectives

---

By the end of this notebook, you will be able to:

1. Understand the basic components of a typical aeroelastic section (airfoil geometry, elastic axis, aerodynamic forces)
2. Derive the equilibrium equation for a wing section with torsional elasticity using moment balance
3. Compute and visualize the **aeroelastic polar**: how the elastic deformation angle varies with flight speed
4. Recognize how the position of the elastic axis relative to the aerodynamic center determines the system behavior
5. Identify the critical question: *What happens when we increase the flight speed further?*

## Introduction: What is Aeroelasticity?

---

**Aeroelasticity** is the study of the interaction between:
- **Aerodynamic forces** (generated by airflow over a structure)
- **Elastic forces** (due to structural flexibility)
- **Inertial forces** (due to mass and acceleration)

In this notebook, we'll focus on a **static aeroelastic problem** where there is no acceleration, thus no inertial forces, and we have a balance between aerodynamic and elastic forces.

### A Simple Example: Why Aircraft Wings Twist

When an aircraft flies, the wings generate lift. This lift creates forces and moments that can **twist** the wing. Because the wing is elastic (not perfectly rigid), it deforms. This deformation changes the angle at which the wing meets the airflow, which in turn changes the aerodynamic forces. 

This is a **feedback loop**: 
- Aerodynamic forces → Elastic deformation
- Elastic deformation → Changed aerodynamics
- Changed aerodynamics → Different forces

In a static aeroelastic problem (no acceleration), we seek an equilibrium where the aerodynamic forces and elastic forces balance out.

Understanding this coupling is essential to analyze aeroelastic instabilities.

## 1. Aerodynamic Model: Lift on an Airfoil

---

Let's start by modeling the aerodynamic forces on a 2D airfoil section. This represents a slice through an aircraft wing, perpendicular to the span direction. Schematically, we can represent the airfoil immersed in a uniform airflow with a given velocity $V$ and at a given angle of attack $\alpha$. This configuration generates an aerodynamic force which can be decomposed into **lift** and **drag** components, respectively perpendicular and parallel to the incoming flow direction.

<figure>
<p align="center">
<img src="../figures/01_airfoil.png" style="width:75%">
<figcaption align = "center"> Airfoil and resultant forces, reproduced from https://mwi-inc.com/blog-post/what-is-an-airfoil-and-whats-its-purpose/. </figcaption>
</p>
</figure>

In many conditions the lift force is much larger than the drag force, and for the purpose of our study we can neglect drag and focus on lift. Therefore, we need to model how much lift the airfoil generates.

We'll use **thin airfoil theory** with the following assumptions:

1. **Incompressible flow**: Air density $\rho$ is constant (valid at low Mach numbers, M < 0.3)
2. **Steady flow**: No time-varying effects (static analysis)
3. **Thin airfoil**: Thickness is small compared to chord
4. **Small angles**: Angle of attack $\alpha$ is small (linear regime)
5. **2D flow**: Infinite span (no 3D effects)

According to this theory, the lift force on the airfoil section is given by:

$$
l = \frac{1}{2} \rho V^2 c c_l
$$

where:
- $\rho$ = air density (kg/m³)
- $V$ = flight velocity (m/s)
- $c$ = airfoil chord length (m)
- $c_l$ = sectional lift coefficient (dimensionless)

We often use the **dynamic pressure**:

$$
q = \frac{1}{2} \rho V^2
$$

This represents the kinetic energy per unit volume of the airflow. With this definition the lift equation becomes:

$$
l = q c c_l
$$

### What is the Lift Coefficient?

The **sectional lift coefficient** ($c_l$) is a dimensionless number that relates the lift generated by an airfoil to the dynamic pressure and the reference chord length. It depends on the airfoil shape and the angle of attack $\alpha$, that is to say the angle between the chord line and the incoming flow (there are other dependencies such as Reynolds and Mach number, but for our study we can neglect those). Restricting our analysis to symmetric airfoils, thin airfoil theory gives us a linear relationship with $\alpha$:

$$
c_l = c_{l\alpha} \alpha
$$

where $c_{l\alpha}$ is the lift curve slope ($\partial c_l$/$\partial\alpha$), in rad<sup>-1</sup>. For the thin airfoil theory, we have $c_{l\alpha} = 2\pi$ rad<sup>-1</sup>.

Given a fixed dynamic pressure (so fixed air density and velocity) and chord length, we can see that the lift varies linearly with angle of attack:

$$
l = q c c_{l\alpha} \alpha
$$

Let's now visualize this linear relationship.

First we import a couple of libraries.
- `numpy` for numerical calculations with vector and matrices (providing functionalities similar to MATLAB)
- `matplotlib` for plotting graphs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Remember that using the import-as syntax allows us to use shorter names for the libraries

# This line enables interactive plots in Jupyter notebooks
# If running in google colab, change "widget" to "inline" (interactive plots don't work in colab and we need to use static images)
%matplotlib widget

First, let's visualize the values of $c_l$ predicted by thin airfoil theory as a function of angle of attack $\alpha$ (in degrees). However, we need to remind ourselves that we have assumed small angles of attack for thin airfoil theory to be valid. So, before plotting the lift coefficient as a function of angle of attack, it makes sense to ask ourselves: *What is a "small" angle of attack?* To visualize this, we can plot the cosine function. The idea is that for small angles, the cosine is close to 1, and the farer the cosine deviates from 1, the less valid our small angle assumption becomes.

In [ ]:
# Define a range of angles from 0 to 90 degrees
angles_deg = np.linspace(0, 90, 100)

# Convert angles to radians (numpy trigonometric functions use radians!)
angles_rad = np.radians(angles_deg)

# Calculate cosine values
cos_values = np.cos(angles_rad)

# Create figure and plot the cosine function
plt.figure()
plt.plot(angles_deg, cos_values)
    
# Add horizontal black line at y=1
plt.axhline(y=1, color='k', linestyle='--')

# Add axis labels, grid, and show the plot
plt.xlabel('$\\alpha$, degrees')  # we can use LaTeX syntax within the $...$ for math
plt.ylabel('$\\cos(\\alpha)$')    # the \\ is needed to escape the backslash in Python strings
plt.grid(True)
plt.show()

If we want to stay within 2% deviation from 1 (careful, this 2% is totally arbitrary!), we can consider angles up to about 12 degrees. Let's define a range of angles of attack between -12 and +12 degrees, computed the corresponding lift values, and plot the results.

In [ ]:
# Define range of angles of attack from -12 to +12 degrees
alpha_deg = np.linspace(-12, 12, 100)

# Convert angles to radians
alpha_rad = np.radians(alpha_deg)

# Define lift curve slope for thin airfoil
c_l_alpha = 2 * np.pi  # per radian

# Calculate lift coefficient
c_l = c_l_alpha * alpha_rad

# Create figure and plot lift coefficient vs angle of attack
plt.figure()
plt.plot(alpha_deg, c_l)

# Add axis labels, grid, and show the plot
plt.xlabel('$\\alpha$, degrees')
plt.ylabel('$c_l$')
plt.grid(True)
plt.show()

As expected, we visualize the linear relationship between lift coefficient and angle of attack. Once again it is worth noticing that $c_l$ is a dimensionless number, and you can see how typical absolute values for an airfoil are in the order of 1 or lower.

As a second step, we can visualize the values of lift that we can obtain for the same range of angles of attack, given a specific flight condition (namely air density and velocity) and chord length. Let's consider the air density at sea level (1.225 kg/m<sup>3</sup>), a speed of 80 m/s, and a chord of 1.6 m, and let's plot the lift as a function of angle of attack.

In [ ]:
# Define air density, speed, and chord
rho = 1.225  # kg/m^3
v = 80       # m/s
c = 1.6      # m

# Calculate dynamic pressure
q = 0.5 * rho * v**2

# Calculate lift per unit span for each angle of attack
lift = q * c * c_l

# Create figure and plot lift vs angle of attack
plt.figure()
plt.plot(alpha_deg, lift)

# Add axis labels, grid, and show the plot
plt.xlabel('$\\alpha$, degrees')
plt.ylabel('Lift per unit span, N/m')
plt.grid(True)
plt.show()

Same linear relationship, but now in terms of lift force instead of lift coefficient. As you can see, typical lift values for an airfoil section can be in the order of thousands of Newtons per meter or more. Bear in mind that this is lift per unit span, as we are only considering a 2D section, and to obtain the total lift for a finite wing you would need to integrate along the span.

However, how does the lift curve look for a real airfoil? We can consider a NACA 0012 airfoil, which is relatively thin (remember, we are using thin airfoil theory), as its maximum thickness amounts to 12% of the chord length. By using the experimental results by Abbott and von Doenhoff available at [this source](https://turbmodels.larc.nasa.gov/naca0012_val.html), we can compare the thin airfoil theory predictions with real life data. Let's copy the $c_l$ data from the website, plot the lift curve and compare it with our previous thin airfoil theory approximation.

In [ ]:
# Just defining an array here with the c_l vs alpha data from Abbott and von Doenhoff for a NACA 0012 airfoil
# Source: https://turbmodels.larc.nasa.gov/naca0012_val.html
abbott_doenhoff_data = np.array([[-17.2794, -1.25323],
                                 [-16.2296, -1.34704],
								 [-15.8616, -1.54416],
								 [-15.1713, -1.51805],
								 [-14.3133, -1.44038],
								 [-13.2811, -1.3712],
								 [-12.2535, -1.25912],
								 [-11.2222, -1.18135],
								 [-10.1947, -1.06927],
								 [-8.14138, -0.827958],
								 [-6.25579, -0.638207],
								 [-5.22822, -0.526128],
								 [-4.19972, -0.422627],
								 [-1.96944, -0.215533],
								 [0., 0.],
								 [0.940006, 0.120611],
								 [1.96944, 0.215533],
								 [2.99515, 0.34477],
								 [3.85131, 0.439599],
								 [4.87888, 0.551678],
								 [5.90831, 0.6466],
								 [7.96346, 0.870758],
								 [10.1891, 1.12074],
								 [11.0471, 1.19842],
								 [13.1088, 1.36252],
								 [16.3759, 1.59591],
								 [16.5678, 1.42443],
								 [17.2971, 1.09024]])

# Create figure and axes
# This time we use the subplots function to get access to the axes object, since we need to plot multiple curves
fig, ax = plt.subplots()

# Calculate thin airfoil theory lift coefficient
# This time we use the same range of angles as in the Abbott and von Doenhoff data for a better visual comparison
lift_thin_airfoil = c_l_alpha * np.radians(abbott_doenhoff_data[:, 0])

# Plot thin airfoil theory lift coefficient
ax.plot(abbott_doenhoff_data[:, 0], lift_thin_airfoil, label='Thin Airfoil Theory')

# Plot Abbott and von Doenhoff data
ax.plot(abbott_doenhoff_data[:, 0], abbott_doenhoff_data[:, 1], 'o-', label='Abbott & von Doenhoff Data')

# Add axis labels, legend, grid, and show the plot
ax.set_xlabel('$\\alpha$, degrees')
ax.set_ylabel('$c_l$')
ax.legend()
ax.grid(True)
plt.show()

As you can see from the plot, thin airfoil theory provides a good approximation of the lift curve for small angles of attack, but as we increase $\alpha$, the real airfoil starts to deviate from the linear behavior. This deviation starts occurring at about $\pm 13^\circ$, and then we  see an abrupt drop in lift at about $\pm 16^\circ$. This is called _stall_ and it is caused by _flow separation_ over the airfoil, which is not captured by the thin airfoil theory.

We don't have to worry about stall for the content of this course, but it is always important to be aware of the limitations of the models we use. Remember that all models are wrong, but some are useful!

## 2. The Typical Section Model

---

To study aeroelastic behavior, we use a simple model called the **typical section**. This is essentially an airfoil where we add some springs (elasticity) and some mass properties. As you can see below, the complete version of the typical section includes three degrees of freedom: heave (up and down), pitch (rotation of the entire airfoil), and control surface (rotation of the flap relative to the main airfoil).

<figure>
<p align="center">
<img src="../figures/01_typical_section_3_dofs.svg" style="width:75%">
<figcaption align = "center">  Typical section with three degrees of freedom: heave, pitch, and control surface. </figcaption>
</p>
</figure>

For this notebook, we'll focus on a simplified version with only **one degree of freedom**, corresponding to the **torsional rotation** or **pitch** degree of freedom, as you can see below.

<figure>
<p align="center">
<img src="../figures/01_typical_section_pitch_dof.svg" style="width:75%">
<figcaption align = "center">  Typical section with pitch degree of freedom only. </figcaption>
</p>
</figure>

### Key Components

On top of the aifoil geometry, our typical section includes:

1. **Elastic axis (EA)**: The point about which the section can rotate, restrained by a torsional spring
2. **Aerodynamic center (AC)**: The point where the lift force is applied (at the quarter-chord, $c/4$ from leading edge, for thin airfoils at low speeds)

### What is the Elastic Axis?

The **elastic axis** is the locus of shear centers along a wing. In turn, for each cross-section of the wing, the shear center is the position where there is zero rotation for a shear load applied to that cross-section. In other words, you can think of it as the "hinge line" for torsional rotation.

In our model, we restrain this rotation with a **torsional spring** of stiffness $k_\theta$ (units: Nm/rad), representing the wing's resistance to twisting. As you will see, the position of the elastic axis relative to the aerodynamic center is crucial for determining the aeroelastic behavior of the typical section. In real wings, the location and the meaning of the elastic axis is a more complex topic (you can read more about it in [this paper](https://doi.org/10.2514/1.C033186)), but for our simplified model we don't have to worry about those complexities.

We define the nondimensional distance $e$ between the elastic axis and the aerodynamic center as the physical distance divided by the chord length.

**Important:** By convention, we define $e$ as positive when the aerodynamic center is ahead of the elastic axis. Thus, when $e > 0$, the lift acts **ahead** of the elastic axis. This configuration, as we'll see, can lead to interesting (and potentially dangerous) behavior.

## 3. The Aeroelastic Coupling

---

Now here's where aeroelasticity enters the picture.

### Two Angles

We need to distinguish between two angles:

1. **$\theta_0$**: The **rigid angle of attack** - the angle the wing would have if it were perfectly stiff (no deformation)
2. **$\theta$**: The **elastic twist angle** - the additional rotation caused by elastic deformation

The **total angle of attack** seen by the airflow is:

$$
\alpha = \theta_0 + \theta
$$

### The Feedback Loop

Imagine to place this configuration in a wind tunnel. Here's what would happen:

1. We set the airfoil at angle $\theta_0$
2. Lift is generated: $L = q c c_{L\alpha} \theta_0$
3. This lift creates a moment about the elastic axis
4. The moment causes the wing to twist by angle $\theta$
5. This twist changes the angle of attack
6. The changed angle of attack changes the lift: $L = q c c_{L\alpha} (\theta_0 + \theta)$
7. Go back to step 3...

The system reaches **equilibrium** when all forces and moments balance.

### Deriving the Equilibrium Equation

Let's find the equilibrium angle $\Theta$ using **moment balance** about the elastic axis.

#### Step 1: Aerodynamic Moment

The lift $L$ acts at the quarter-chord point (aerodynamic center). The moment arm about the elastic axis is the distance $ec\cos\alpha$. However, since we are assuming small angles, we can approximate $\cos\alpha \approx 1$, so the moment arm is simply $ec$.

The aerodynamic moment about the elastic axis (nose-up positive) is:

$$
M_a = L \cdot ec = q c c_{l\alpha} \alpha \cdot ec = q e c^2 c_{l\alpha} (\theta_0 + \theta)
$$

As you can see from this equation, when $e > 0$ (AC ahead of EA), the lift creates a nose-up moment that tends to increase the total angle of attack (which in turn increases lift and moment further).

#### Step 2: Elastic Restoring Moment

The torsional spring provides a restoring moment:

$$
M_e = -k_\theta \theta
$$

The negative sign indicates that the spring **opposes** the rotation, meaning that for a positive (nose-up) twist angle, the spring generates a negative (nose-down) moment.

#### Step 3: Equilibrium Condition

At equilibrium, the sum of moments about the elastic axis is zero:

$$
M_a + M_e = 0
$$

$$
q e c^2 c_{l\alpha} (\theta_0 + \theta) -k_\theta \theta = 0
$$

#### Step 4: Solve for the Elastic Angle $\theta$

Expanding:

$$
q e c^2 c_{l\alpha} \theta_0 + q e c^2 c_{l\alpha} \theta - k_\theta \theta = 0
$$

Collecting terms with $\theta$:

$$
\theta (q e c^2 c_{l\alpha} - k_\theta) = -q e c^2 c_{l\alpha} \theta_0
$$

Therefore:

$$
\boxed{\theta = \frac{-q e c^2 c_{l\alpha} \theta_0}{q e c^2 c_{l\alpha} - k_\theta} = \frac{q e c^2 c_{l\alpha}}{k_\theta - q e c^2 c_{l\alpha}} \theta_0}
$$

This is our **fundamental aeroelastic equation** for the typical section! It tells us the equilibrium elastic twist angle $\theta$ given certain properties of the typical section ($k_\theta$, $e$, $c$) and the flight conditions ($q$, $\theta_0$).

Let's examine this result:

$$
\theta = \frac{q e c^2 c_{l\alpha}}{k_\theta - q e c^2 c_{l\alpha}} \theta_0
$$

**Key observations:**

1. **Proportionality:** $\theta$ is proportional to $\theta_0$. Doubling the initial angle doubles the elastic twist.

2. **Speed dependence:** $\theta$ depends on dynamic pressure $q = \frac{1}{2}\rho V^2$, so it increases with flight speed squared.

3. **The denominator matters:** 
   - At low speeds: $k_\theta \gg q e c^2 c_{l\alpha}$, so $\theta \approx 0$ (rigid behavior)
   - As speed increases: denominator decreases, $\theta$ increases
   - **What happens when the denominator approaches zero?** 🤔 
   - **What happens when $e < 0$ (AC behind EA)?** 🤔

We'll explore these questions later! For now, let's have a look at the effects of this result on the lift curve.

## 4. The Aeroelastic Polar

---

Recall that for a rigid airfoil, the lift varies linearly with angle of attack:

$$
l_\text{rigid} = q c c_{l\alpha} \theta_0
$$

However, for an elastic airfoil, the total angle of attack is $\alpha = \theta_0 + \theta$, where $\theta$ depends on the dynamic pressure through our aeroelastic equation. Therefore, the actual lift becomes:

$$
l = q c c_{l\alpha} (\theta_0 + \theta) = q c c_{l\alpha} \left(\theta_0 + \frac{q e c^2 c_{l\alpha}}{k_\theta - q e c^2 c_{l\alpha}} \theta_0\right)
$$

Factoring out $\theta_0$:

$$
l = q c c_{l\alpha} \theta_0 \left(1 + \frac{q e c^2 c_{l\alpha}}{k_\theta - q e c^2 c_{l\alpha}}\right)
$$

Simplifying the expression in parentheses:

$$
l = q c c_{l\alpha} \theta_0 \left(\frac{k_\theta - q e c^2 c_{l\alpha} + q e c^2 c_{l\alpha}}{k_\theta - q e c^2 c_{l\alpha}}\right)
$$

$$
\boxed{l = \frac{q c c_{l\alpha} k_\theta}{k_\theta - q e c^2 c_{l\alpha}}\theta_0}
$$

This is the **aeroelastic polar equation**. It shows how lift varies with the rigid angle of attack $\theta_0$ taking into account the elastic deformation of the airfoil.

### Understanding the Aeroelastic Polar

Let's compare this with the rigid case:

- **Rigid airfoil:** $l_\text{rigid} = q c c_{l\alpha} \theta_0$
- **Elastic airfoil:** $l = q c c_{l\alpha} \frac{k_\theta}{k_\theta - q e c^2 c_{l\alpha}}\theta_0$

The key difference is the term $k_\theta/\left(k_\theta - q e c^2 c_{l\alpha}\right)$. This term modifies the lift slope due to aeroelastic effects. Is this term greater than or less than 1? In other words, does it increase or decrease the lift slope? Let's explore this further.

The term $k_\theta/\left(k_\theta - q e c^2 c_{l\alpha}\right)$ can be either greater than 1 or less than 1, depending on whether the quantity $q e c^2 c_{l\alpha}$ is positive or negative, respectively. However, the only parameter that can be either positive or negative is $e$, the nondimensional distance between the elastic axis and the aerodynamic center.

Let's calculate this in practice. We consider a typical section with a torsional stiffness of $k_\theta = 5\cdot 10^4$ Nm/rad, and three possible positions of the elastic axis:
1. **Elastic axis behind aerodynamic center:** $e = -0.15$
2. **Elastic axis at aerodynamic center:** $e = 0$
3. **Elastic axis ahead of aerodynamic center:** $e = 0.15$

For the the chord, air density, and flight speed, we keep the same values as before. Let's calculate the term $\frac{k_\theta}{k_\theta - q e c^2 c_{l\alpha}}$ in these three cases and print the results.

In [ ]:
# Define torsional spring stiffness
k_theta = 5e4  # torsional spring stiffness [Nm/rad]

# Define nondimensional distances between elastic axis and aerodynamic center
# We define the three cases as an array
e_range = np.array([0.15, 0.0, -0.15])

# Calculate the term k_theta / (K_theta - q * e * c^2 * c_{l\alpha}) for each case
lift_slope_factor_range = k_theta / (k_theta - q * e_range * c**2 * c_l_alpha)

# Print the results
for e, lift_slope_factor in zip(e_range, lift_slope_factor_range):  # zip allows us to iterate over two arrays in parallel
    print(f"e/c = {e:.2f}, Lift slope factor = {lift_slope_factor:.3f}")  # we use formatted string literal (f-string) for nice output

See? We have previously mentioned that when the elastic axis is ahead of the aerodynamic center (positive e), the lift generates a moment that tends to increase the angle of attack, leading to a larger elastic twist and thus a higher lift. This is reflected in the lift slope factor being greater than 1. Conversely, when the elastic axis is behind the aerodynamic center (negative e), the lift generates a moment that reduces the angle of attack, leading to a smaller elastic twist and thus a lower lift. This is reflected in the lift slope factor being less than 1. When the elastic axis coincides with the aerodynamic center (e = 0), there is no aeroelastic effect on the lift slope, as no moment is generated by the lift about the elastic axis, so the lift slope factor is exactly 1.

Let's visualize this effect by comparing the elastic lift curves (aeroelastic polars) obtained for $e = -0.15$ and $0.15$ with the rigid lift curve (the case $e = 0$ is equivalent to the rigid case).

In [ ]:
# Define range of rigid angles of attack from -12 to +12 degrees
theta_0_deg = np.linspace(-12, 12, 100)

# Convert angles to radians
theta_0_rad = np.radians(theta_0_deg)

# Calculate lift of rigid airfoil
l_rigid = q * c * c_l_alpha * theta_0_rad

# Lift for elastic airfoil with e > 0
# We can simply multiply the rigid lift by the lift slope factor calculated previously
l_elastic_e_positive = l_rigid * lift_slope_factor_range[0]  # remember that in python indexes start at 0

# Lift for elastic airfoil with e < 0
l_elastic_e_negative = l_rigid * lift_slope_factor_range[2]

# Create figure and axes
fig, ax = plt.subplots()

# Plot the lift curves
ax.plot(theta_0_deg, l_rigid, label='Rigid airfoil')
ax.plot(theta_0_deg, l_elastic_e_positive, label=f'$e>0$ (AC ahead of EA)')
ax.plot(theta_0_deg, l_elastic_e_negative, label=f'$e<0$ (AC behind EA)')

# Add axis labels, legend, grid, and show the plot
ax.set_xlabel('$\\theta_0$, degrees')
ax.set_ylabel('Lift per unit span, N/m')
ax.legend()
ax.grid(True)
plt.show()

Here you have it! As you can see, when the elastic axis is ahead of the aerodynamic center ($e>0$), the lift curve is steeper than the rigid case, indicating a higher lift slope due to aeroelastic effects. Conversely, when the elastic axis is behind the aerodynamic center ($e<0$), the lift curve is less steep, showing a reduced lift slope.

At this point, you probably have noticed that term $k_\theta - q e c^2 c_{l\alpha}$ appearing in the denominator of both the elastic twist angle $\theta$ and the lift $l$. What happens when, as we increase the flight speed and thus the dynamic pressure, this term approaches zero? That's what we'll explore in the next section!

## 5. Torsional Divergence

---

In the previous section, we saw that for $e > 0$, the denominator of both the elastic twist angle and the lift decreases as we increase the speed and thus the dynamic pressure. This leads to an increase in both $\theta$ and $l$. Let's now have a look at what happens when the denominator reaches zero.

Rather than looking at $\theta$ directly, let's examine the ratio $\theta/\theta_0$, which represents the elastic deformation normalized by the rigid angle of attack. This ratio tells us: *By what factor does the elastic deformation amplify the initial angle?*

From our aeroelastic equation:

$$
\frac{\theta}{\theta_0} = \frac{q e c^2 c_{l\alpha}}{k_\theta - q e c^2 c_{l\alpha}}
$$

Let's plot this ratio as a function of dynamic pressure for a positive $e$ value (elastic axis ahead of aerodynamic center). We define a range of flight speeds, compute the corresponding dynamic pressures, and then calculate the ratio $\theta/\theta_0$ for each dynamic pressure value. We keep all other parameters unchanged.

In [ ]:
# Define nondimensional distance between elastic axis and aerodynamic center
e = 0.15  # [dimensionless]

# Create range of flight speeds from 0 to 230 m/s
v_range = np.linspace(0, 230, 500)  # [m/s]

# Calculate dynamic pressures
q_range = 0.5 * rho * v_range**2  # [Pa]

# Calculate theta/theta_0 for the varying dynamic pressures
theta_ratio_range = (q_range * e * c**2 * c_l_alpha) / \
                    (k_theta - q_range * e * c**2 * c_l_alpha)  # [dimensionless]

# Create figure and plot the ratio
plt.figure()
plt.plot(q_range, theta_ratio_range)

# Add axis labels, grid, and show the plot
plt.xlabel('$q$, Pa')
plt.ylabel('$\\theta/\\theta_0$')
plt.grid(True)
plt.show()

From the plot above, we can see that as we approach a value of 23.5 kPa, the ratio $\theta/\theta_0$ increases dramatically, until the term $k_\theta - q e c^2 c_{l\alpha}$ becomes negative and the ratio becomes negative as well. Of course, a negative ratio doesn't make physical sense in this context, and what we are actually observing is a vertical asymptote in the plot. In physical terms, this asymptote represents the condition of **torsional divergence**. This is a static aeroelastic instability that occurs when the aerodynamic moment generated by the lift overcomes the elastic restoring moment provided by the torsional spring. As you can imagine, this instability can lead to a catastrophic structural failure, so it's something that we definitely want to avoid in real aircraft design!

The value of dynamic pressure at which this asymptote occurs is called the **divergence dynamic pressure** ($q_D$), and it can be calculated by setting the term $k_\theta - q e c^2 c_{l\alpha}$ to zero and solving for $q$:

$$
k_\theta - q_D e c^2 c_{l\alpha} = 0
$$

$$\boxed{q_D = \frac{k_\theta}{e c^2 c_{l\alpha}}}$$

Since $q = \frac{1}{2}\rho V^2$, we can also find the **divergence speed** ($V_D$):

$$
\frac{1}{2}\rho V_D^2 = q_D
$$

$$
\boxed{V_D = \sqrt{\frac{2 q_D}{\rho}} = \sqrt{\frac{2 k_\theta}{\rho e c^2 c_{l\alpha}}}}
$$

Let's compute the divergence dynamic pressure and divergence speed for our typical section parameters.

In [ ]:
# Calculate divergence dynamic pressure
q_divergence = k_theta / (e * c**2 * c_l_alpha)  # [Pa]

# Calculate divergence speed
v_divergence = np.sqrt(2 * q_divergence / rho)  # [m/s]

# Print results
print(f"Divergence dynamic pressure: {q_divergence:.2f} Pa")
print(f"Divergence speed: {v_divergence:.2f} m/s")

Let's now consider the parameters that influence the torsional divergence of the typical section. As you can see from the equation of the divergence dynamic pressure, divergence does not depend on the rigid angle of attack $\theta_0$, but only on the geometrical ($c$), structural ($k_\theta$, $e$), and aerodynamic ($c_{l\alpha}$) properties of the system.

Since the chord and the lift curve slope are usually parameters that can't be changed easily, we focus our attention on the torsional stiffness $k_\theta$ and the elastic axis position $e$ (which can be modified through structural design choices). Let's visualize how the divergence speed changes when we vary the torsional stiffness.

In [ ]:
# Define range of torsional stiffness values
k_theta_range = np.linspace(1e4, 1e5, 100)  # [Nm/rad]

# Calculate divergence speeds for each torsional stiffness value
# We keep all other parameters same as before, with elastic axis ahead of aerodynamic center (e > 0)
v_divergence_range = np.sqrt(2 * (k_theta_range / (e * c**2 * c_l_alpha)) / rho)  # [m/s]

# Create figure and plot divergence speed vs torsional stiffness
plt.figure()
plt.plot(k_theta_range, v_divergence_range)

# Add axis labels, grid, and show the plot
plt.xlabel('$k_\\theta$, Nm/rad')
plt.ylabel('$V_D$, m/s')
plt.grid(True)
plt.show()

As we could imagine, increasing the torsional stiffness leads to a higher divergence speed, meaning that the structure can withstand higher flight speeds before reaching the divergence condition. Consequently, if we want to push the divergence speed higher, and thus increase the range of safe operating speeds, making the structure stiffer in torsion is a viable strategy. However, this typically comes at the cost of increased weight, which is always a critical consideration in aircraft design.

Now, let's explore how the position of the elastic axis affects the divergence speed. Recall that $e$ is defined as positive when the aerodynamic center is ahead of the elastic axis. Therefore, as we move the elastic axis closer to the aerodynamic center (decreasing $e$), we expect the divergence speed to increase, since the moment arm for the lift-induced moment decreases. And what happens when the $e$ becomes negative and the elastic axis is behind the aerodynamic center? Let's visualize this effect by plotting the divergence speed as a function of $e$.

In [ ]:
# Define range of nondimensional distances between elastic axis and aerodynamic center
# Remember that e is the distance nondimensionalized by the chord length
# The aerodynamic center is located at the quarter-chord point, so when considering positive e it does not make sense to go beyond 0.75 and when considering negative e it does not make sense to go beyond -0.25
e_range = np.linspace(-0.2, 0.7, 100)  # [dimensionless]

# Calculate divergence speeds for each e value
v_divergence_range = np.sqrt(2 * (k_theta / (e_range * c**2 * c_l_alpha)) / rho)  # [m/s]

# Create figure and plot divergence speed vs e
plt.figure()
plt.plot(e_range, v_divergence_range)

# Add axis labels, grid, and show the plot
plt.xlabel('$e$')
plt.ylabel('$V_D$, m/s')
plt.grid(True)
plt.show()

What is happening here? The cell returned the following warning:

```
RuntimeWarning: invalid value encountered in sqrt
  v_divergence_range = np.sqrt(2 * (k_theta / (e_range * c**2 * c_l_alpha)) / rho)  # [m/s]
```

and in the figure we see the curve plotted only for positive values of $e$. This is because when $e$ becomes negative, the entire term under the square root in the divergence speed equation becomes negative, leading to an undefined (imaginary) divergence speed. Physically speaking, this means that when the elastic axis is behind the aerodynamic center, the system cannot experience torsional divergence. In fact, in this configuration, the lift generates a moment that tends to reduce the angle of attack, providing a stabilizing effect rather than a destabilizing one. Therefore, the system remains stable for all flight speeds, and no divergence occurs.

You may be thinking: _"cool, let's design all aircraft wings with the elastic axis behind the aerodynamic center so that we can forget about divergence!"_ Unfortunately, this would be too good to be true. In reality, placing the elastic axis ahead of the aerodynamic center is simply not possible because of structural and design constraints. However, avoiding to have the elastic axis too far behind of the aerodynamic center is something that surely helps in preventing divergence.

## 6. Exercise: Effect of Altitude on Divergence Speed

In the last plots of the notebook, we computed the divergence speed assuming a constant air density corresponding to sea level conditions (1.225 kg/m<sup>3</sup>). However, airplanes do not fly at sea level! They typically cruise at altitudes between 10,000 m and 12,000 m, where the air density is significantly lower. This reduction in air density affects the divergence speed.

Since air density $\rho$ decreases with altitude, the divergence speed **increases** at higher altitudes. This is good news, and now it's your turn to explore this effect!

#### How Air Density Varies with Altitude

The **International Standard Atmosphere (ISA)** model gives us the variation of air properties with altitude. You can see an illustration of this model below.

<figure>
<img src="../figures/01_atmospheric_properties.png" style="width:100%">
<figcaption align = "center"> Variation of air properties according to ISA, reproduced from https://eaglepubs.erau.edu/introductiontoaerospaceflightvehicles/chapter/international-standard-atmosphere-isa/. </figcaption>
</figure>

For altitudes up to about 11 km (troposphere), the temperature decreases linearly with altitude:

$$
T(h) = T_0 - L h
$$

where:
- $T_0 = 288.15$ K (sea level temperature)
- $L = 0.0065$ K/m (temperature lapse rate)
- $h$ = altitude [m]

The air density is given by:

$$
\rho(h) = \rho_0 \left(\frac{T(h)}{T_0}\right)^{\frac{g}{R L} - 1}
$$

where:
- $\rho_0 = 1.225$ kg/m³ (sea level density)
- $g = 9.81$ m/s² (gravitational acceleration)
- $R = 287$ J/(kg·K) (specific gas constant for air)

**Your task:**

1. First, plot altitude vs. air density from sea level to 11,000 m
2. Then, plot altitude vs. divergence speed for the same range
3. Comment on the results: Why does divergence become less of a concern at high altitudes?

You can use the same typical section parameters as before, or experiment with different values, up to you!

In [ ]:
# Physical constants for ISA model
T_0 = 288.15  # sea level temperature [K]
L = 0.0065  # temperature lapse rate [K/m]
g = 9.81  # gravitational acceleration [m/s^2]
R = 287  # specific gas constant for air [J/(kg·K)]
rho_0 = 1.225  # sea level density [kg/m³]

# Altitude range
h_array = np.linspace(0, 11000, 100)  # [m] from sea level to 11 km

# TODO: Calculate temperature at each altitude
# T_array = ...

# TODO: Calculate air density at each altitude using ISA model
# rho_array = ...

# TODO: Calculate divergence speed at each altitude
# V_D_array_altitude = ...


# TODO: Plot 1 - Altitude vs air density
# plt.figure()
# ...

# TODO: Plot 2 - Altitude vs divergence speed
# plt.figure()
# ...

# TODO: Print some values
# print(f"At sea level (h = 0 m):")
# print(f"Air density: ... kg/m³")
# print(f"Divergence speed: ... m/s")
# print(f"\n")  # just a blank line
# print(f"At altitude h = 11000 m:")
# print(f"Air density: ... kg/m³")
# print(f"Divergence speed: ... m/s")
# print(f"\n")
# print(f"Divergence speed increased by a factor of ...")

## 6. Summary and Key Takeaways

---

In this notebook, we've explored two static aeroelastic phenomena of the typical section:

### 1. The Aeroelastic Polar

The **aeroelastic polar** describes how the lift slope of an airfoil changes when accounting for elastic deformation:

$$
l = q c c_{l\alpha} \frac{k_\theta}{k_\theta - q e c^2 c_{l\alpha}} \theta_0
$$

**Key insights:**
- For $e > 0$ (AC ahead of EA): positive feedback between moment and rotation leads to steeper lift curve
- For $e = 0$ (AC at EA): no aeroelastic coupling, behaves as rigid
- For $e < 0$ (AC behind EA): negative feedback between moment and rotation leads to a smaller lift slope

### 2. Torsional Divergence

**Torsional divergence** is a static aeroelastic instability that occurs when:

$$
k_\theta = q_D e c^2 c_{l\alpha}
$$

The **divergence speed** is:

$$
V_D = \sqrt{\frac{2 k_\theta}{\rho e c^2 c_{l\alpha}}}
$$

**Key insights:**
- Only occurs for $e > 0$ (typical wing configuration)
- Increasing $k_\theta$ (torsional stiffness) increases $V_D$ ✓
- Increasing $e$ (elastic axis further behind AC) decreases $V_D$ ✗
- Lower air density (higher altitude) increases $V_D$ ✓

### Design Implications

To prevent divergence, aircraft designers:
1. Can maximize torsional stiffness $k_\theta$ through structural design, although this may increase weight
2. Can control elastic axis position relative to aerodynamic center
3. Must ensure $V_D$ exceeds maximum operating speeds with adequate margin across the different altitudes

### Looking Ahead

In the next notebook, we'll extend our model to include:
- **Two degrees of freedom:** torsion AND plunge (heave) motion
- **Dynamic effects:** inertial forces and time-dependent behavior
- **Flutter:** A dynamic aeroelastic instability even more dangerous than divergence

Get ready to dive into **eigenvalue analysis**!

---

**End of Notebook 1** ✓

*Continue to Notebook 2: Flutter Analysis with the 2-DOF Typical Section*